In [1]:
# standard imports
import pandas as pd
import numpy as np
from collections import Counter

# plotting bits
import matplotlib.pyplot as plt
import os

pd.set_option('display.max_rows', 2500)
pd.set_option('display.max_columns', 100)

# stats packages to fit classification models
import statsmodels.api as sm
import statsmodels.formula.api as smf
from sklearn.model_selection import train_test_split
from sklearn import metrics
from sklearn.calibration import calibration_curve

# importing xG xGils library
import xGils.xG as xG

# importing seaborn to get colourblind palette for calibration plots
import seaborn
palette = seaborn.color_palette('colorblind', 6).as_hex()

pd.options.mode.chained_assignment = None

# **xG Feature Engineering**

1. Load in pre-made Opta dataset.
2. Load in synthetic data.
2. Feature engineering of additional features:
    * "Basic" features;
    * "Added" features;
    * "Advanced" features;
    * "Advanced" features including synthetic shots.
3. Fit logistic / probit regression models.
4. Plot calibration curves.
5. Calculate model scores.
6. Apply xT to Opta dataset.
7. Output dataset.

**Note, we'll have to construct some of the features before we construct a `df_shots` dataframe.**

**Will also want to integrate the synthetic shots and see if that improves things.**
(Will have to generate some additional synthetic data for the synthetic shot dataset using appropriate distributions).

Take a look at this: https://twitter.com/Soccermatics/status/1260593848973172744?s=20

## **1) Loading Opta dataset (which includes Bayesian xT)**

In [2]:
%%time

df = pd.read_csv('/Users/christian/Desktop/University/Birkbeck MSc Applied Statistics/Project/Data/Analysis Ready/Opta Bayesian xT/Bayesian_Opta_xT.csv')

# converting the timestamp string to a datetime
df['timeStamp'] = pd.to_datetime(df.timeStamp, format='%Y-%m-%d %H:%M:%S.%f')
df['kickOffDateTime'] = pd.to_datetime(df.kickOffDateTime, format='%Y-%m-%d %H:%M:%S.%f')

print (f'{len(df)} rows loaded.\n')

3126182 rows loaded.

CPU times: user 14.8 s, sys: 1.41 s, total: 16.2 s
Wall time: 16.4 s


### **Setting useful event types (may not need this)**

In [3]:
# pass events (inc. crosses)
opta_successful_pass_events = ['2nd Assist','Assist','Chance Created','Cross','Pass']
opta_failed_pass_events = ['Failed Pass','Offside Pass']

# dribble events
opta_successful_dribble_events = ['Dribble']
opta_failed_dribble_events = ['Failed Dribble']

# shot events
opta_successful_shot_events = ['Goal']
opta_failed_shot_events = ['Hit Woodwork','Miss','Missed Penalty','Penalty Saved','Shot Blocked','Shot Saved']

opta_events_successful = opta_successful_pass_events + opta_successful_dribble_events + opta_successful_shot_events
opta_events_relevant = opta_successful_pass_events + opta_failed_pass_events + opta_successful_dribble_events + opta_failed_dribble_events + opta_successful_shot_events + opta_failed_shot_events

opta_events_relevant

['2nd Assist',
 'Assist',
 'Chance Created',
 'Cross',
 'Pass',
 'Failed Pass',
 'Offside Pass',
 'Dribble',
 'Failed Dribble',
 'Goal',
 'Hit Woodwork',
 'Miss',
 'Missed Penalty',
 'Penalty Saved',
 'Shot Blocked',
 'Shot Saved']

## **2) Loading in Synthetic Shot Data**

In [4]:
df_synthetic = pd.read_csv('/Users/christian/Desktop/University/Birkbeck MSc Applied Statistics/Project/Data/Synthetic/Synthetic_Shots.csv')

## **3) Feature Engineering**

#### Binary Response Variable
* Shot success = 1 (`goalScoredFlag`)

#### Simple Features:
* Initial $x$-distance from goal (in metres), (`x_dist_goal`)
* Initial $y$-distance from the middle of the pitch (in metres), (`c1_m`)

#### Added Features:
* Angle from shooting position to the centre of the goal, (`a`)
* Amount of goal shooter can see, (`aShooting`)
* Distance to goal (metres), (`D`)
* Squared distance to goal (metres$^2$), (`Dsquared`)

#### Advanced (Contextual) Features:
* Game state (the point-in-time difference in goals between the two sides), (`goalDelta`)
* Headcount difference due to red cards (e.g. is equal to 1 if 11 Vs 10), (`numReds`)
* Player possession duration, (`playerPossessionTimeSec`)
* Cumulative team possession sequence duration, (`possessionTimeSec`)
* Binary penalty flag (`penaltyFlag`)
* Count of defenders providing defensive pressure on the shot (`pressureCountOnShot`)

### Contextual Features as Proxies:
* Using cumulative team possession sequence duration as a proxy for a counter attack / how set the opposing team is in defence.
* Using player possession duration as a proxy for how quickly the player took the shot after receiving the ball.
* Count of defenders applying shot pressure a proxy for how open the shot is.

**Didn't include previous action as 2nd assists and assists are by definition going to result in a goal, whereas passes and crosses aren't. Don't have separate labels.**

### **Applying Contextual Feature Engineering Functions**

In [5]:
%%time

df = xG.xG_contextual_feature_engineering(df)

CPU times: user 5min 19s, sys: 13.4 s, total: 5min 32s
Wall time: 3min 32s


### **Applying Geometric Feature Engineering Functions**

In [ ]:
%%time

df_shots = xG.xG_geometric_shot_feature_engineering(df)

Processed 0 shots out of 43813
Processed 1000 shots out of 43813
Processed 2000 shots out of 43813
Processed 3000 shots out of 43813
Processed 4000 shots out of 43813
Processed 5000 shots out of 43813
Processed 6000 shots out of 43813
Processed 7000 shots out of 43813
Processed 8000 shots out of 43813
Processed 9000 shots out of 43813
Processed 10000 shots out of 43813
Processed 11000 shots out of 43813
Processed 12000 shots out of 43813
Processed 13000 shots out of 43813
Processed 14000 shots out of 43813
Processed 15000 shots out of 43813
Processed 16000 shots out of 43813
Processed 17000 shots out of 43813
Processed 18000 shots out of 43813
Processed 19000 shots out of 43813
Processed 20000 shots out of 43813
Processed 21000 shots out of 43813
Processed 22000 shots out of 43813
Processed 23000 shots out of 43813
Processed 24000 shots out of 43813
Processed 25000 shots out of 43813
Processed 26000 shots out of 43813
Processed 27000 shots out of 43813
Processed 28000 shots out of 4381

### **Saving & Loading the Engineered Shot Data**

In [ ]:
df.to_csv('/Users/christian/Desktop/University/Birkbeck MSc Applied Statistics/Project/Data/Analysis Ready/Opta Engineered Shots/Engineered Events.csv', index=None)
df_shots.to_csv('/Users/christian/Desktop/University/Birkbeck MSc Applied Statistics/Project/Data/Analysis Ready/Opta Engineered Shots/Engineered Shots.csv', index=None)

In [ ]:
df_shots.read_csv('/Users/christian/Desktop/University/Birkbeck MSc Applied Statistics/Project/Data/Analysis Ready/Opta Engineered Shots/Engineered Shots.csv')

# converting the timestamp string to a datetime
df_shots['timeStamp'] = pd.to_datetime(df_shots.timeStamp, format='%Y-%m-%d %H:%M:%S.%f')
df_shots['kickOffDateTime'] = pd.to_datetime(df_shots.kickOffDateTime, format='%Y-%m-%d %H:%M:%S.%f')

---

### **Adding in the synthetic data**

* Want to create randomly sampled data for the new contextual features:
    * goalDelta
    * numReds
    * possessionTimeSec
    * playerPossessionTimeSec
* And will then push it through the geometric function to add the geometric features.

In [ ]:
# adding some flags to enable differentiation between real and synthetic shots
df_synthetic['eventType'] = 'shot'
df_synthetic['realOrSynthetic'] = 'synthetic'
df_synthetic['goalScoredFlag'] = df_synthetic.goal

# synthesising goalDelta: sampling from normal distribution and rounding to nearest integer
df_synthetic['goalDelta'] = np.round(np.random.normal(loc=df_shots.goalDelta.mean(), scale=df_shots.goalDelta.std(), size=len(df_shots)), 0).astype(int)

# synthesising numReds: sampling from normal and rounding to nearest integer
df_synthetic['numReds'] = np.round(np.random.normal(loc=df_shots.numReds.mean(), scale=df_shots.numReds.std(), size=len(df_shots)), 0).astype(int)

# synthesising possessionTimeSec by simulating from exponential with scale = mean (possessionTimeSec is a lifetime)
df_synthetic['possessionTimeSec'] = np.random.exponential(scale=df_shots.possessionTimeSec.mean(), size=len(df_shots))

# synthesising playerPossessionTimeSec by simulating from an exponential with scale = mean (playerPossessionTimeSec is a lifetime)
df_synthetic['playerPossessionTimeSec'] = np.random.exponential(scale=df_shots.playerPossessionTimeSec.mean(), size=len(df_shots))

# penaltyFlag, setting to 0 for all synthetic shots because all synthetic shots are taken outside the box
df_synthetic['penaltyFlag'] = 0

# pressureCountOnShot, setting these to 0, too, given where the synthetic shots are being taken from
df_synthetic['pressureCountOnShot'] = 0

# passing the synthetic shots dataframe through the geometric shot feature engineering function
## dropping the "goal" column so that we can easily concatenate the synthetic shots with the real shots
df_synthetic_enriched = xG.xG_geometric_shot_feature_engineering(df_synthetic).drop(columns='goal')

# concatenating real and synthetic shots
## starting by just copying the real shots
df_shots_incl_synthetic = df_shots.copy()
## marking real shots as real
df_shots_incl_synthetic['realOrSynthetic'] = 'real'
## then mixing real and synthetic shots
df_shots_incl_synthetic = pd.concat([df_shots_incl_synthetic[df_synthetic_enriched.columns], df_synthetic_enriched]).reset_index(drop=True).copy()

### **Model Fitting**

#### Splitting `df_shots` into training and test dataset, stratifying the dependent variable

In [ ]:
# splitting into a dataframe for training and dataframe for testing
## stratifying the successFlag
df_shots_train, df_shots_test = train_test_split(df_shots, test_size=0.2, stratify=df_shots.goalScoredFlag, random_state=0, shuffle=True)

print (f'Stratified Shot Success Rates:\n\nOverall: {100*np.round(df_shots.goalScoredFlag.mean(),3)}%\nTrain: {100*np.round(df_shots_train.goalScoredFlag.mean(), 3)}%\nTest: {100*np.round(df_shots_test.goalScoredFlag.mean(), 3)}%\n')

### **Fitting basic model to training data**:

In [ ]:
%%time

xG_model_basic = smf.glm(formula="goalScoredFlag ~ x_dist_goal + c1_m", data=df_shots_train\
                 ,family=sm.families.Binomial()).fit()

xG_model_basic.summary2()

### **Fitting additional features:**

In [ ]:
%%time

xG_model_added = smf.glm(formula="goalScoredFlag ~  x_dist_goal + c1_m + a + aShooting + D + Dsquared", data=df_shots_train\
                 ,family=sm.families.Binomial()).fit()

xG_model_added.summary2()

### **Fitting model to training data with advanced features:**


In [ ]:
%%time

xG_formula_adv = "goalScoredFlag ~  x_dist_goal +\
                    c1_m + a + aShooting + D +\
                    Dsquared + possessionTimeSec + playerPossessionTimeSec +\
                    goalDelta + numReds  + D*a + C(penaltyFlag) + pressureCountOnShot"

xG_model_adv = smf.glm(formula=xG_formula_adv, data=df_shots_train\
                 ,family=sm.families.Binomial()).fit()

xG_model_adv.summary2()

### **Fitting same advanced model but using data including *synthetic shots*.**

In [ ]:
df_shots_train, df_shots_test = train_test_split(df_shots_incl_synthetic, test_size=0.4, stratify=df_shots_incl_synthetic.goalScoredFlag, random_state=0, shuffle=True)

# now getting rid of the synthetic shots from the test data
df_shots_test = df_shots_test.loc[df_shots_test['realOrSynthetic'] ==  'real'].reset_index(drop=True).copy()

print (f'Stratified Shot Success Rates:\n\nOverall: {100*np.round(df_shots.goalScoredFlag.mean(),3)}%\nTrain: {100*np.round(df_shots_train.goalScoredFlag.mean(), 3)}%\nTest: {100*np.round(df_shots_test.goalScoredFlag.mean(), 3)}%\n')

In [ ]:
%%time

xG_model_adv_syn = smf.glm(formula=xG_formula_adv, data=df_shots_train\
                 ,family=sm.families.Binomial()).fit()

xG_model_adv_syn.summary2()

---

### **Applying *four* xG models to *test* data**

In [ ]:
df_shots_test = xG.apply_xG_model_to_test(df_shots_test, [xG_model_basic, xG_model_added, xG_model_adv, xG_model_adv_syn])

---


### **Model Validation: Calibration Curves of Models Fit to Test Data**

### Calibration Curve: Basic Vs Added Models

In [ ]:
def plot_calibration_curve(df_shots_test, numBins=25, alpha=0.6, saveOutput=0, plotName='xG_calibration_plot'):
    """
    Calibration plots for xG models
    """
    fig = plt.figure(figsize=(10, 15))
    
    # splitting figure into two subplots
    gs = fig.add_gridspec(ncols=1, nrows=2, height_ratios=(2/3, 1/3))
    
    # defining axes of subplots
    ax1 = fig.add_subplot(gs[0])
    ax2 = fig.add_subplot(gs[1])

    # getting colourbline palette
    palette = seaborn.color_palette('colorblind', 6).as_hex()

    # Plotting perfect calibration (line y=x)
    ax1.plot([0, 1], [0, 1], 'k:', label='Perfectly Calibrated Model', alpha=alpha)

    # FOUR calibration curves - Tricky to plot all four at a time, so just do a Simple Vs Advanced
    ## 1) Simple Model
    fraction_of_positives, mean_predicted_value = calibration_curve(df_shots_test.goalScoredFlag, df_shots_test.xG_basic, n_bins=numBins)
    ax1.plot(mean_predicted_value, fraction_of_positives, marker="o", markersize=10, label='Basic Model', alpha = alpha, lw=1, color=palette[4])

    ## 2) Added Model
    fraction_of_positives, mean_predicted_value = calibration_curve(df_shots_test.goalScoredFlag, df_shots_test.xG_added, n_bins=numBins)
    ax1.plot(mean_predicted_value, fraction_of_positives, marker="o", markersize=10, label='Added Features', alpha = alpha, lw=2, color=palette[2])

    ## 3) Advanced Model: Canonical (Logit) Link function
    fraction_of_positives, mean_predicted_value = calibration_curve(df_shots_test.goalScoredFlag, df_shots_test.xG_adv, n_bins=numBins)
    ax1.plot(mean_predicted_value, fraction_of_positives, marker="o", markersize=10, label='Advanced Features', alpha = alpha, lw=3, color=palette[1])

    ## 4) Advanced Model: Using Synthetic data to train BUT NOT TEST
    fraction_of_positives, mean_predicted_value = calibration_curve(df_shots_test.goalScoredFlag, df_shots_test.xG_syn, n_bins=numBins)
    ax1.plot(mean_predicted_value, fraction_of_positives, marker="o", markersize=10, label='Advanced Features + Synthetic Shots', alpha=alpha, lw=4, color=palette[0])

    ax1.set_title('Calibration Plot', fontsize=20, pad=10)
    ax1.set_ylabel('Fraction of Successful Test Shots', fontsize=16)
    ax1.set_xlabel('Mean xG', fontsize=16)

    ax1.set_ylim([-0.05, 1.05])
    ax1.set_xlim([-0.05, 1.05])

    ax1.legend(loc="lower right", fontsize=16)

    ax1.tick_params(labelsize=16)

    # now plotting histogram
    seaborn.distplot(df_shots_test.xG_syn, color=palette[0], label='Advanced Features + Synthetic Shots', kde=False, ax=ax2)
    
    ax2.set_title('Distribution of xG Test Predictions (Real Shots Only)', fontsize=20, pad=10)
    ax2.set_ylabel('Number of Test Shots', fontsize=16)
    ax2.set_xlabel('Predicted xG', fontsize=16)
    ax2.tick_params(labelsize=16)
    ax2.legend(fontsize=16)

    plt.tight_layout()

    if saveOutput == 1:
        plt.savefig(f'/Users/christian/Desktop/University/Birkbeck MSc Applied Statistics/Project/Plots/xG Calibration/{plotName}.pdf', dpi=300, format='pdf', bbox_inches='tight')

    return plt.show()

In [ ]:
plot_calibration_curve(df_shots_test, alpha=0.5, saveOutput=1, plotName='xG calibration incl synthetic shots')

---

### **Applying Logistic Regression Classifier and Calculating Model Fit Metrics**

In [20]:
xG.calculate_model_metrics(df_shots_test, 'xG_syn')

Brier Score: 0.07649732734909391

Precision Score: 0.6991150442477876

Recall Score: 0.08342133051742344

F1 Score: 0.1490566037735849

AUC Score: 0.5395533048526203

AccuracyScore: 0.8978135266795061


---

### **Applying xG Model (*Trained Using Advanced Features + Synthetic Shots*) to All Real Shots** 

In [ ]:
df_shots['xG'] = xG_model_adv_syn.predict(df_shots_test)

### **Creating a Simple Dataframe `df_xG` Just Containing `eventId` & `xG` Columns**

In [ ]:
df_xG = df_shots[['eventId','xG']].reset_index(drop=True).copy()

### **And Now Joining `df_xG` Back OnTo Original `df`.**

> **All non-shots will get an xG of 0**

> `excess_xG` defined as `goalScoredFlag` - `xG`

In [173]:
eventId = 538
df.loc[(df['eventId'] >= eventId-5)& (df['eventId'] <= eventId+5)][['playerName','playerTeamId','eventType','eventSubType','gameTime','timeStamp','possessionTeamId','homeTeamId','awayTeamId','goalScoredFlag']]

,playerName,playerTeamId,eventType,eventSubType,gameTime,timeStamp,possessionTeamId,homeTeamId,awayTeamId,goalScoredFlag
532,Alexandre Lacazette,3,attack,Chance Created,26:21,2017-08-11 20:12:25.790,3,3,13,0
533,Sead Kolasinac,3,shot,Shot Saved,26:22,2017-08-11 20:12:26.407,3,3,13,0
534,Kasper Schmeichel,13,defence,Save,26:22,2017-08-11 20:12:26.507,3,3,13,0
535,Granit Xhaka,3,attack,Failed Pass,26:41,2017-08-11 20:12:45.128,3,3,13,0
536,Matty James,13,defence,Clearance,26:43,2017-08-11 20:12:47.364,13,3,13,0
537,Mohamed Elneny,3,shot,Miss,26:44,2017-08-11 20:12:48.479,3,3,13,0
538,Danny Simpson,13,press,Pressure on Shot,26:44,2017-08-11 20:12:48.479,3,3,13,0
539,Matty James,13,press,Pressure on Shot,26:44,2017-08-11 20:12:48.479,3,3,13,0
540,Wes Morgan,13,press,Pressure on Shot,26:44,2017-08-11 20:12:48.479,3,3,13,0
541,Kasper Schmeichel,13,attack,Failed Pass,27:13,2017-08-11 20:13:16.554,13,3,13,0


In [210]:
df_shots.columns

Index(['competition', 'season', 'seasonIndex', 'gameMonthIndex', 'matchId',
       'playerId', 'playerName', 'position', 'detailedPosition',
       'playerTeamId', 'minsPlayed', 'subIn', 'subOut',
       'replacedReplacingPlayerId', 'booking', 'eventId', 'eventType',
       'eventSubType', 'eventTypeId', 'x1', 'y1', 'x2', 'y2', 'gameTime',
       'timeStamp', 'periodId', 'homeTeamName', 'homeTeamId', 'awayTeamName',
       'awayTeamId', 'kickOffDateTime', 'minute', 'second', 'x1_m', 'y1_m',
       'x2_m', 'y2_m', 'possessionTeamId', 'possessionSequenceIndex',
       'possessionStartTime', 'possessionTimeSec', 'playerPossessionTimeSec',
       'goalDelta', 'numReds', 'goalScoredFlag', 'xT', 'x_dist_goal',
       'x_dist_goal_2', 'c1_m', 'c1_m_2', 'vec_x', 'vec_y', 'D', 'Dsquared',
       'Dcubed', 'a', 'aShooting'],
      dtype='object')

In [215]:
%%time

# storing list of counts of pressures on shots
lst_pressureCountOnShot = []

# getting list of all shot eventId's
shot_eventIds = df_shots.eventId.values[:10]

# iterating through shots
for n, shot in enumerate(shot_eventIds):
    
    if n % 1000 == 0:
        print (f'Processed {n} shots out of {len(shot_eventIds)}')
    
    # getting shot meta to query the full event dataframe with: matchId,  teamId, timestamp
    matchId, teamId, timeStamp = df_shots.loc[df_shots['eventId'] == shot, ['matchId','playerTeamId','timeStamp']].values[0]
    
    # producing a dataframe of pressures on each shot, where the pressure must be coming from a player on the other team to the one taking a shot
    # and of course must be happening in the same match
    # and has been recorded two seconds either side of the shot (this is the timedelta logic)
    ## THIS IS A VERY EXPENSIVE QUERY TO RUN
    df_shotPressure = df.loc[(df['eventSubType'] == 'Pressure on Shot') & (df['playerTeamId'] != teamId) & (df['matchId'] ==  matchId) &\
                  (df['timeStamp'] > timeStamp - pd.Timedelta(2,'s')) & (df['timeStamp'] < timeStamp + pd.Timedelta(2,'s'))]
            
    # and we set the pressureCountOnShot metric to simply be the count of the players that are applying pressure to the shot
    lst_pressureCountOnShot.append(len(df_shotPressure))
    
# updating df_shots dataframe all at once
df_shots['pressureCountOnShot'] = lst_pressureCountOnShot
    

Processed 0 shots out of 10
CPU times: user 1.92 s, sys: 16.3 ms, total: 1.94 s
Wall time: 1.76 s


In [202]:
%%time

# storing  0/1 list of penalties
df_shots['penaltyFlag'] = 0

# getting subset of of all shot eventId's that could be penalties due to the location of the shot (92.925,34.0)
shot_eventIds = df_shots.loc[(df_shots['x1_m'] == 92.925) & (df_shots['y1_m'] == 34.000)].eventId.values

# iterating through shots
for shot in shot_eventIds:
    
    # getting shot meta to query the full event dataframe with: matchId,  teamId, timestamp
    matchId, teamId, timeStamp = df_shots.loc[df_shots['eventId'] == shot, ['matchId','playerTeamId','timeStamp']].values[0]
    
    # producing a dataframe of pressures on each shot, where the pressure must be coming from a player on the other team to the one taking a shot
    # and of course must be happening in the same match
    # and has been recorded 5 minutes before the shot
    df_penalty = df.loc[(df['playerTeamId'] != teamId) & (df['matchId'] ==  matchId) &\
                  (df['timeStamp'] > timeStamp - pd.Timedelta(300,'s')) & (df['timeStamp'] < timeStamp) &\
                  (df['eventSubType'].isin(['Conceded Penalty','Foul for Penalty']))]
    
    # and we set the pressureCountOnShot metric to simply be the count of the players that are applying pressure to the shot
    df_shots.loc[df_shots['eventId'] == shot, 'penaltyFlag'] = 1 if len(df_penalty) > 0 else 0

CPU times: user 54.6 s, sys: 568 ms, total: 55.2 s
Wall time: 47.7 s


In [191]:
eventId = 14265
df.loc[(df['eventId'] >= eventId-5)& (df['eventId'] <= eventId+5)][['playerName','playerTeamId','eventType','eventSubType','gameTime','timeStamp','possessionTeamId','homeTeamId','awayTeamId','goalScoredFlag','x1_m','y1_m']]

,playerName,playerTeamId,eventType,eventSubType,gameTime,timeStamp,possessionTeamId,homeTeamId,awayTeamId,goalScoredFlag,x1_m,y1_m
14259,Emre Can,14,attack,Pass,53:45,2017-08-12 13:43:10.683,14,57,14,0,70.455,53.516
14260,Joel Matip,14,attack,Pass,53:50,2017-08-12 13:43:15.412,14,57,14,0,57.435,38.352
14261,Roberto Firmino,14,attack,Pass,53:51,2017-08-12 13:43:16.620,14,57,14,0,79.065,33.728
14262,Heurelho Gomes,57,defence,Foul for Penalty,53:55,2017-08-12 13:43:20.714,14,57,14,0,10.080,42.432
14263,Mohamed Salah,14,attack,Fouled,53:55,2017-08-12 13:43:20.714,14,57,14,0,94.920,25.568
14264,Heurelho Gomes,57,defence,Conceded Penalty,54:40,2017-08-12 13:44:04.935,14,57,14,0,0.000,36.924
14265,Roberto Firmino,14,shot,Goal,54:40,2017-08-12 13:44:04.935,14,57,14,1,92.925,34.000
14266,Tom Cleverley,57,attack,Pass,55:14,2017-08-12 13:44:39.128,57,57,14,0,52.395,33.524
14267,Miguel Britos,57,attack,Pass,55:19,2017-08-12 13:44:44.201,57,57,14,0,26.040,43.520
14268,Stefano Okaka,57,attack,Failed Pass,55:23,2017-08-12 13:44:47.913,57,57,14,0,65.835,44.948
